# Integração

* Requer:
    * `./data/tpr_data_parsed.csv`;
    * `./data/co2_data_parsed.csv`.
* Saída em:
    * `./out/final`.

<br />

* Verificar `parse.ipynb` antes para instruções;

<br />

* Une datasets pelo critério de distância mínima:
    1. Para cada ponto de tempetura, obtém a sua data e encontra todos os pontos de $CO_2$ nessa mesma data;
    2. Calcula a distância entre o ponto de temperatura e todos os outros pontos de $CO_2$ anteriormente selecionados;
    3. Escolhe aquele de mínima distância como o ponto equivalente entre os *datasets*.

<br />

* Cálculo da distância:
    * ✔️ **Abordagem 1:** plana — $d(P_1, P_2) = \sqrt{(x_1-x_2)^2 + (y_1-y_2)^2}$;
    * ❓ **Abordagem 2:** esférica — Faz sentido? É necessária? Como fazer?

<br />

* Sobre a implementação:
    * Em um cenário ideial, a integração seria feita utilizando `CROSS JOIN` e aplicando à risca o método;
        * Tal alternativa é inviável, haja vista que geraria registros numa ordem de grandeza de $10^{6 + 8}$.
    * A título de usababilidade, foi feita a integração utilizando `LEFT JOIN` e adequação dos domínios dos valores de latitude e longitude.
        * Em síntese, as coordenadas do *dataset* de temperatura passam a ser discretizadas tal como as presentes no de $CO_2$;
        * Ambos os métodos possuem um exato mesmo *result set*.

# Bibliotecas e Configurações

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.config(
        "spark.jars.packages",
        "io.xskipper:xskipper-core_2.12:1.3.0"
).getOrCreate()

In [2]:
from xskipper import Xskipper # Necessáriamente após build do spark

In [3]:
metadata_path = "./tmp/metadata"

config = dict([
    ("io.xskipper.parquet.mdlocation", metadata_path),
    ("io.xskipper.parquet.mdlocation.type", "EXPLICIT_BASE_PATH_LOCATION")
])

Xskipper.setConf(spark, config)

# Leitura

In [4]:
tpr_parsed_path = './data/tpr_data_parsed.csv'
co2_parsed_path = './data/co2_data_parsed.csv'

tpr_reader = spark.read.options(header='True').format("csv")
co2_reader = spark.read.options(header='True').format("csv")

tpr_data = tpr_reader.load(tpr_parsed_path)
co2_data = co2_reader.load(co2_parsed_path)

# Otimização

* [Xskipper](https://xskipper.io/1.3.0/) é uma biblioteca que busca trazer uma indexação (alguma coisa) próxima a do SQL no Spark;
* Em síntese, fornece opções para indexação de colunas baseada em critérios, por exemplo:
    * Colunas com poucos dados únicos, mas muitos dados: fazer listagem dos valores únicos;
    * Colunas com ordenação crescente/decrescente: obter valores limítrofes;
    * Entre outras.
* Todos esses metadados levantados a respeitos das colunas agilizam as operações (reduzem tempo de execução) pois permitem pular registros e registros

In [5]:
tpr_xskipper = Xskipper(spark, tpr_parsed_path)

if tpr_xskipper.isIndexed(): tpr_xskipper.dropIndex()

tpr_xskipper.indexBuilder()                 \
            .addValueListIndex("t_date")    \
            .addValueListIndex("lon_grid")  \
            .addValueListIndex("lat_grid")  \
            .build(tpr_reader)              \
            .show(10, False)

+-------+-----------------+-------------------+
|status |new_entries_added|old_entries_removed|
+-------+-----------------+-------------------+
|SUCCESS|1                |0                  |
+-------+-----------------+-------------------+



In [6]:
co2_xskipper = Xskipper(spark, co2_parsed_path)

if co2_xskipper.isIndexed(): co2_xskipper.dropIndex()

co2_xskipper.indexBuilder()              \
            .addValueListIndex("c_date") \
            .addValueListIndex("c_lon")  \
            .addValueListIndex("c_lat")  \
            .build(co2_reader)           \
            .show(10, False)

+-------+-----------------+-------------------+
|status |new_entries_added|old_entries_removed|
+-------+-----------------+-------------------+
|SUCCESS|1                |0                  |
+-------+-----------------+-------------------+



# Integração

In [7]:
if not Xskipper.isEnabled(spark): Xskipper.enable(spark)

tpr_data.createOrReplaceTempView("tpr_data")
co2_data.createOrReplaceTempView("co2_data")

merged_data = spark.sql(
    """
    SELECT
        t_date AS date,
        t_lat AS lat,
        t_lon AS lon,
        city,
        country,
        tpr,
        tpr_unc,
        co2
    FROM tpr_data
    LEFT JOIN co2_data
    ON
        t_date = c_date AND
        lon_grid = c_lon AND
        lat_grid = c_lat
    """
)

A escrita dos dados leva até 6 minutos e resulta em, aproximadamente, 500 MB. Descomentar se necessário.

In [8]:
# merged_data.coalesce(1).write.format('csv').option('header', True).mode('overwrite').save('./out/final')